In [ ]:
import os
import cv2
import face_recognition
from skimage.feature import hog
from skimage import exposure
from google.colab.patches import cv2_imshow
import numpy as np
from sklearn import svm

# Set up paths for known and unknown faces
known_faces_dir = '/content/gdrive/MyDrive/images/known_faces/'
unknown_faces_dir = '/content/gdrive/MyDrive/images/unknown_faces/'

# Load known faces
known_faces = []
known_face_names = []
for filename in os.listdir(known_faces_dir):
    if os.path.isfile(os.path.join(known_faces_dir, filename)):
        name = os.path.splitext(filename)[0]
        image = face_recognition.load_image_file(known_faces_dir + filename)
        try:
            # Normalize the image
            image = cv2.normalize(image, None, 0, 255, cv2.NORM_MINMAX)
            encoding = face_recognition.face_encodings(image)[0]
            known_faces.append(encoding)
            known_face_names.append(name)
            print(f"face detected in {filename}")
        except IndexError:
            print(f"No face detected in {filename}")

# Train SVM classifier
svm_clf = svm.SVC(kernel='linear', probability=True)
svm_clf.fit(known_faces, known_face_names)

# Process unknown faces
for filename in os.listdir(unknown_faces_dir):
    image = face_recognition.load_image_file(unknown_faces_dir + filename)
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Detect face locations using HOG algorithm
    face_locations = face_recognition.face_locations(gray_image, model='hog')

    # Calculate feature vector using HOG algorithm
    features, hog_image = hog(gray_image, orientations=8, pixels_per_cell=(16, 16), cells_per_block=(1, 1), visualize=True, multichannel=False)

    # Adjust the gamma value for exposure control
    gamma = 0.38  # adjust as needed
    gamma_corrected = np.uint8(cv2.pow(hog_image / 255., gamma) * 255.)

    # Predict the name of the recognized face using SVM classifier
    for (top, right, bottom, left) in face_locations:
        face_encoding = face_recognition.face_encodings(image, [(top, right, bottom, left)])[0]
        name = svm_clf.predict([face_encoding])[0]

        # Draw a rectangle around the recognized face and show the name
        cv2.rectangle(image, (left, top), (right, bottom), (0, 255, 0), 2)
        cv2.putText(image, name, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        print(f"face recognized in {filename} as {name}")

    # Show the image with the recognized faces
    cv2_imshow(image)
    cv2.waitKey(0)

# Clean up
cv2.destroyAllWindows()